# Pandas

## Лабораторная работа №2

### Базовые операции с `DataFrame`

1.1 В файлах `recipes_sample.csv` и `reviews_sample.csv` находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца с индексами в таблице `reviews` (безымянный столбец).

In [ ]:
import pandas as pd

recipes = pd.read_csv('data/recipes_sample.csv')
reviews = pd.read_csv('data/reviews_sample.csv', index_col=0)

1.2 Для каждой из таблиц выведите основные параметры:
* количество точек данных (строк);
* количество столбцов;
* тип данных каждого столбца.

In [ ]:
# For the recipes table:
print("Recipes table:")
print("Number of rows:", recipes.shape[0])
print("Number of columns:", recipes.shape[1])
print("Data types:")
print(recipes.dtypes)

# For the reviews table:
print("\nReviews table:")
print("Number of rows:", reviews.shape[0])
print("Number of columns:", reviews.shape[1])
print("Data types:")
print(reviews.dtypes)

1.3 Исследуйте, в каких столбцах таблиц содержатся пропуски. Посчитайте долю строк, содержащих пропуски, в отношении к общему количеству строк.

In [ ]:
# For the recipes table:
print("Recipes table:")
for col in recipes.columns:
    if recipes[col].isna().sum() > 0:
        print(f"Column {col} has {recipes[col].isna().mean():.2%} missing values.")
    else:
        print(f"Column {col} has no missing values.")

# For the reviews table:
print("\nReviews table:")
for col in reviews.columns:
    if reviews[col].isna().sum() > 0:
        print(f"Column {col} has {reviews[col].isna().mean():.2%} missing values.")
    else:
        print(f"Column {col} has no missing values.")

1.4 Рассчитайте среднее значение для каждого из числовых столбцов (где это имеет смысл).

In [ ]:
# For the recipes table:
numeric_cols = recipes.select_dtypes(include=['int64', 'float64']).columns.drop(['id', 'contributor_id'])
print("Recipes table:")
for col in numeric_cols:
    print(f"Average value for {col}: {recipes[col].mean()}")

# For the reviews table:
numeric_cols = reviews.select_dtypes(include=['int64', 'float64']).columns.drop(['user_id', 'recipe_id'])
print("\nReviews table:")
for col in numeric_cols:
    print(f"Average value for {col}: {reviews[col].mean()}")

1.5 Создайте серию из 10 случайных названий рецептов.

In [ ]:
import numpy as np

# Create a series of 10 random recipe names
random_names = recipes['name'].sample(n=10)
print(random_names)

1.6 Измените индекс в таблице `reviews`, пронумеровав строки, начиная с нуля.

In [ ]:
# Change the index in the reviews table
reviews = reviews.reset_index(drop=True)
print(reviews.head())

1.7 Выведите информацию о рецептах, время выполнения которых не больше 20 минут и кол-во ингредиентов в которых не больше 5.

In [ ]:
# Filter recipes that take no more than 20 minutes and have no more than 5 ingredients
quick_and_easy = recipes[(recipes['minutes'] <= 20) & (recipes['n_ingredients'] <= 5)]
print(quick_and_easy[['name', 'minutes', 'n_ingredients']])

### Работа с датами в `pandas`

2.1 Преобразуйте столбец `submitted` из таблицы `recipes` в формат времени. Модифицируйте решение задачи 1.1 так, чтобы считать столбец сразу в нужном формате.

In [ ]:
# Load data with 'submitted' column as datetime
recipes = pd.read_csv('data/recipes_sample.csv', parse_dates=['submitted'])
print(recipes.info())

2.2 Выведите информацию о рецептах, добавленных в датасет не позже 2010 года.

In [ ]:
# Filter recipes added to the dataset no later than 2010
old_recipes = recipes[recipes['submitted'].dt.year <= 2010]
print(old_recipes[['name', 'submitted']])

### Работа со строковыми данными в `pandas`

3.1  Добавьте в таблицу `recipes` столбец `description_length`, в котором хранится длина описания рецепта из столбца `description`.

In [ ]:
recipes['description_length'] = recipes['description'].str.len()
print(recipes.head())

3.2 Измените название каждого рецепта в таблице `recipes` таким образом, чтобы каждое слово в названии начиналось с прописной буквы.

In [ ]:
recipes['name'] = recipes['name'].str.title()
print(recipes.head())

3.3 Добавьте в таблицу `recipes` столбец `name_word_count`, в котором хранится количество слов из названии рецепта (считайте, что слова в названии разделяются только пробелами). Обратите внимание, что между словами может располагаться несколько пробелов подряд.

In [ ]:
recipes['name_word_count'] = recipes['name'].str.split().str.len()
print(recipes.head())

### Группировки таблиц `pd.DataFrame`

4.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [ ]:
# Count the number of recipes submitted by each contributor
recipe_counts = recipes['contributor_id'].value_counts()

# Determine which contributor added the maximum number of recipes
max_contributor = recipe_counts.idxmax()

# Output the resulting information
print("Number of recipes submitted by each contributor:\n")
print(recipe_counts)
print("\nContributor who added the most recipes: {}\n".format(max_contributor))

4.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы? Обратите внимание, что отзыв с нулевым рейтингом или не заполненным текстовым описанием не считается отсутствующим.

In [ ]:
# Calculate the average rating for each recipe
avg_ratings = reviews.groupby('recipe_id')['rating'].mean()

# Count the number of recipes with missing reviews
missing_reviews = recipes['id'].nunique() - avg_ratings.count()

# Output the resulting information
print("Average rating for each recipe:\n")
print(avg_ratings)
print("\nNumber of recipes with missing reviews: {}\n".format(missing_reviews))

4.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [ ]:
# Count the number of recipes by year of creation
recipes_by_year = recipes.groupby(recipes['submitted'].dt.year).size()

# Output the resulting information
print("Number of recipes by year of creation:\n")
print(recipes_by_year)

### Объединение таблиц `pd.DataFrame`

5.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты, на которые не оставлен ни один отзыв, должны отсутствовать в полученной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [ ]:
# Merge recipes and reviews dataframes on 'id' and 'recipe_id' columns
merged = pd.merge(recipes[['id', 'name']], reviews[['recipe_id', 'user_id', 'rating']], left_on='id', right_on='recipe_id')

# Drop the 'recipe_id' column
merged.drop('recipe_id', axis=1, inplace=True)

# Output the resulting dataframe
print(merged)

In [ ]:
# если у рецепта нет отзывов, он не появится в результирующей матрице данных
no_reviews_recipe = recipes[recipes['id'] == 222261]

# Check if the recipe is present in the merged dataframe
print(merged[merged['id'] == 222261])

5.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`, где столбец `review_count` содержит кол-во отзывов, оставленных на рецепт `recipe_id`. У рецептов, на которые не оставлен ни один отзыв, в столбце `review_count` должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и найдя строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [ ]:
# Merge recipes and reviews dataframes on 'id' and 'recipe_id' columns
merged = pd.merge(recipes[['id', 'name']], reviews[['recipe_id']], left_on='id', right_on='recipe_id')

# Group by 'recipe_id' and count the number of reviews for each recipe
review_counts = merged.groupby('recipe_id')['recipe_id'].count()

# Create a new dataframe with 'recipe_id', 'name', and 'review_count' columns
recipe_review_counts = pd.DataFrame({'recipe_id': review_counts.index, 'review_count': review_counts.values})

# Merge the new dataframe with the 'recipes' dataframe on 'id' column to get the recipe names
recipe_review_counts = pd.merge(recipe_review_counts, recipes[['id', 'name']], left_on='recipe_id', right_on='id')

# Drop the 'id' and 'recipe_id' columns
recipe_review_counts.drop(['id', 'recipe_id'], axis=1, inplace=True)

# Set the index to be the 'name' column
recipe_review_counts.set_index('name', inplace=True)

# Output the resulting dataframe
print(recipe_review_counts)

In [ ]:
# Select a recipe with no reviews
no_reviews_recipe = recipes[recipes['id'] == 222261]

# Check if the recipe is present in the recipe_review_counts dataframe
print(recipe_review_counts.loc[no_reviews_recipe['name']])

5.3. Выясните, рецепты, добавленные в каком году, имеют наименьший средний рейтинг?

In [ ]:
# Merge recipes and reviews dataframes on 'id' and 'recipe_id' columns
merged = pd.merge(recipes[['id', 'name', 'submitted']], reviews[['recipe_id', 'rating']], left_on='id', right_on='recipe_id')

# Group by year and recipe and calculate the mean rating
grouped = merged.groupby([merged['submitted'].dt.year, 'name'])['rating'].mean()

# Find the year and recipe with the lowest average rating
lowest_rating = grouped.idxmin()

# Output the resulting information
print("Year and recipe with the lowest average rating: {}\n".format(lowest_rating))

### Сохранение таблиц `pd.DataFrame`

6.1 Отсортируйте таблицу в порядке убывания величины столбца `name_word_count` и сохраните результаты выполнения заданий 3.1-3.3 в csv файл. 

In [ ]:
# Add a 'description_length' column to the 'recipes' dataframe
recipes['description_length'] = recipes['description'].apply(lambda x: len(str(x).split()))

# Change the name of each recipe so that each word starts with a capital letter
recipes['name'] = recipes['name'].apply(lambda x: x.title())

# Add a 'name_word_count' column to the 'recipes' dataframe
recipes['name_word_count'] = recipes['name'].apply(lambda x: len(str(x).split()))

# Sort the 'recipes' dataframe in descending order of the 'name_word_count' column
recipes_sorted = recipes.sort_values(by='name_word_count', ascending=False)

# Save the 'recipes' dataframe to a csv file
recipes_sorted.to_csv('recipes_sorted.csv', index=False)

In [ ]:
# Add a 'description_length' column to the 'recipes' dataframe
recipes['description_length'] = recipes['description'].apply(lambda x: len(str(x).split()))

# Change the name of each recipe so that each word starts with a capital letter
recipes['name'] = recipes['name'].apply(lambda x: x.title())

# Add a 'name_word_count' column to the 'recipes' dataframe
recipes['name_word_count'] = recipes['name'].apply(lambda x: len(str(x).split()))

# Sort the 'recipes' dataframe in descending order of the 'name_word_count' column
recipes_sorted = recipes.sort_values(by='name_word_count', ascending=False)

# Output information about recipes that take no more than 20 minutes and have no more than 5 ingredients
quick_and_simple_recipes = recipes[(recipes['minutes'] <= 20) & (recipes['n_ingredients'] <= 5)]
quick_and_simple_recipes.to_csv('recipes_sorted.csv', mode='a', index=False, header=False)

# Change the name of each recipe so that each word starts with a capital letter
recipes['name'] = recipes['name'].apply(lambda x: x.title())

# Add a 'name_word_count' column to the 'recipes' dataframe
recipes['name_word_count'] = recipes['name'].apply(lambda x: len(str(x).split()))

# Save the 'recipes' dataframe to a csv file
recipes.to_csv('recipes_sorted.csv', mode='a', index=False, header=False)

# Output the resulting information
print("Quick and simple recipes:\n")
print(quick_and_simple_recipes)

6.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 5.1 и 5.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 5.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 5.2.

In [ ]:
# Merge the 'recipes' and 'reviews' dataframes on 'id' and 'recipe_id'
merged = pd.merge(recipes[['id', 'name', 'contributor_id']], reviews[['recipe_id', 'user_id', 'rating']], left_on='id', right_on='recipe_id')

# Drop rows with missing reviews
merged = merged.dropna(subset=['rating'])

# Group the merged dataframe by recipe and calculate the number of reviews
review_count = merged.groupby(['id', 'name'])['rating'].count().reset_index()

# Rename the 'rating' column to 'review_count'
review_count = review_count.rename(columns={'rating': 'review_count'})

# Save the results of task 5.1 to a sheet called 'Recipes with grades'
with pd.ExcelWriter('recipes_and_review_scores.xlsx') as writer:
    merged[['id', 'name', 'user_id', 'rating']].to_excel(writer, sheet_name='Recipes with grades', index=False)

# Save the results of task 5.2 to a sheet called 'Number of reviews on recipes'
with pd.ExcelWriter('recipes_and_review_scores.xlsx', mode='a') as writer:
    review_count[['id', 'name', 'review_count']].to_excel(writer, sheet_name='Number of reviews on recipes', index=False)